<h1>IBM Applied AI and Data Science Capstone Project</h1>

In [80]:
import pandas as pd
import numpy as np
import requests
import csv
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


In [81]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #requests.get(url).text will ping a website and return you HTML of the website.

We begin by reading the source code for a given web page and creating a BeautifulSoup (soup)object with the BeautifulSoup function. Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping. Prettify() function in BeautifulSoup will enable us to view how the tags are nested in the document.

In [133]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

our first task is to find class "wikitable sortable" in the HTML script.

In [134]:
My_table = soup.find('table',{'class':'wikitable sortable'})# We can get this by inspecting the Wikipedia page

Now to extract all the links within 'tr tag', we will use find_all().

In [135]:
rows = My_table.find_all('tr')

In [85]:
header = [th.text.rstrip() for th in rows[0].find_all('th')]

with open('output.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)
    for row in rows[1:]:
        data = [td.text.rstrip() for td in row.find_all('td')]
        writer.writerow(data)


The first row ussually contains the header cells. We serch throught the first row in the rows list to get the text values of all th elements in that row.
we also ensure to remove the all trailing whitespaces in the text using the <b>rstrip</b> python string method.
The w mode is used to ensure the file is open for writing.
First we write the header row, then loop through the rest of the rows ignoring the first row to get the data contained within and write the data for all those rows to the file object.

In [86]:
df = pd.read_csv('output.csv')
df = df[df['Borough']!='Not assigned']#process only those rows where 'Borough' is not 'Not assigned'
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [87]:
df1 = pd.DataFrame(df.groupby('Postal Code')['Postal Code'].count()>1)
df1.reset_index(drop=True, inplace=True)#Dropping the index column of a pandas.DataFrame removes the index column from the DataFrame and replaces it with the standard sequential indexing.
df1[df1['Postal Code']==True]

,Postal Code


From above code we found out that there are no repeating postal codes.

In [88]:
df.shape

(103, 3)

Now we are going to fetch the geospatial data for the given postal codes.

In [89]:
!wget -O geodata.csv http://cocl.us/Geospatial_data

--2020-05-08 18:23:36--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.72.228, 159.8.69.24
Connecting to cocl.us (cocl.us)|159.8.69.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-05-08 18:23:37--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-08 18:23:40--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-08 18:23:40--  https://ibm.box.com/public/static/

In [90]:
geodata = pd.read_csv('geodata.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [91]:
df_final = pd.merge(df, geodata, on = 'Postal Code', how = 'inner')#merge the postal data with the geospatial data using inner join on 'Postal Code' column
df_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Now we use geopy library to get the latitude and longitude values of Toronto

In [92]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


We create a map of Toronto with neighbourhoods superimposed on top of it.

In [93]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)#what appears in the label for each marker is set here.
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.4).add_to(map_toronto)  
    
map_toronto

Now we filter only those boroughs which contain the word <b>'Toronto'</b>

In [94]:
borough_names = list(df_final.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

We create a dataframe with only those boroughs that contain <b>'Toronto'</b>

In [95]:
toronto_df_new = df_final[df_final['Borough'].isin(borough_with_toronto)].reset_index(drop=True)#filter out those borough values which are in "borough_with_toronto" list using isin and put the result in a dataframe
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Now we create a map of Toronto with only the filtered out boroughs.

In [96]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.4).add_to(map_toronto)  
    
map_toronto

**We are going to use the Foursquare API to explore the neighborhoods**

In [97]:
CLIENT_ID = 'Your Foursquare Client_ID' # your Foursquare ID
CLIENT_SECRET = 'Your Foursquare Client_Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FEEYOBCQ2CUUGEHQOI3MYRJ3Y5TOQS5TBR0LBRK2ZSSA1N3M
CLIENT_SECRET:BGII3YAOJMHSZY0JJJUNGP0BSLHCFQ4OWQSI2ZO13TZVPE1B


Now, let's get the top 100 venues that are within a radius of 500 meters.

In [98]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Postal Code'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']   #store values of 'items' key in results list 
    for result in results:
        venues.append((
            post, #these values are obtained from outer loop over each row of toronto_df_new
            borough,
            neighborhood,
            lat, 
            long, 
            result['venue']['name'], # these values are obtained from each object in results list
            result['venue']['location']['lat'], 
            result['venue']['location']['lng'],  
            result['venue']['categories'][0]['name']))
        
print(venues[0:3])

[('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Roselle Desserts', 43.653446723052674, -79.3620167174383, 'Bakery'), ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Tandem Coffee', 43.65355870959944, -79.36180945913513, 'Coffee Shop'), ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Morning Glory Cafe', 43.653946942635294, -79.36114884214422, 'Breakfast Spot')]


In [99]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1625, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check the number of entries obtained for each Postal Code

In [100]:
venues_df.groupby(["Postal Code", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postal Code,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",22,22,22,22,22,22
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,23,23,23,23,23,23
M4S,Central Toronto,Davisville,38,38,38,38,38,38
M4T,Central Toronto,"Moore Park, Summerhill East",1,1,1,1,1,1


**Let's find out how many unique categories can be curated from all the returned venues**

In [101]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 233 uniques categories.


In [102]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Coffee Shop', 'Breakfast Spot', 'Distribution Center',
       'Spa', 'Restaurant', 'Park', 'Gym / Fitness Center',
       'Historic Site', 'Farmers Market', 'Chocolate Shop', 'Pub',
       'Performing Arts Venue', 'Dessert Shop', 'French Restaurant',
       'Yoga Studio', 'Café', 'Theater', 'Event Space', 'Ice Cream Shop',
       'Shoe Store', 'Mexican Restaurant', 'Art Gallery',
       'Asian Restaurant', 'Cosmetics Shop', 'Bank', 'Electronics Store',
       'Beer Store', 'Hotel', 'Health Food Store', 'Antique Shop',
       'Sushi Restaurant', 'Italian Restaurant', 'Creperie', 'Beer Bar',
       'Arts & Crafts Store', 'Burrito Place', 'Hobby Shop', 'Diner',
       'Fried Chicken Joint', 'Discount Store', 'Japanese Restaurant',
       'Burger Joint', 'Juice Bar', 'Sandwich Place', 'Gym', 'Bar',
       'College Auditorium', 'College Cafeteria',
       'Vegetarian / Vegan Restaurant'], dtype=object)

<h3>Analyzing Each Area of Dataset</h3>

In [103]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="",prefix_sep="")# To make Venue category values into columns with appropriate names and fill them with dummy values

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Postal Code'] = venues_df['Postal Code'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head(10)

(1625, 236)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [120]:
toronto_grouped = toronto_onehot.groupby(["Postal Code", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(39, 236)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.25000,0.0,0.0,0.0,0.0,0.000,0.0,0.00000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.000,0.0,0.02381
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000,0.0,0.00000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.025,0.0,0.02500
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000,0.0,0.00000


Now let's create the new dataframe and display the top 10 venues for each Postal Code.


In [121]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']#for 1st, 2nd and 3rd

# create columns according to number of top venues
areaColumns = ['Postal Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))#when indicators[ind] exceeds limits,it throws an error on which code in except is executed, i.e.,after 3rd..
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):#loop through all indices(rows) of toronto_grouped(as toronto_grouped.shape[0] gives 1st value of the tuple which the number of rows)
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:] #for each row get all columns from 3rd onwards i.e., all the categories columns
    row_categories_sorted = row_categories.sort_values(ascending=False) #sort in descending to get top values
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues] #fill values of categories with top 10 venues
    
# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted


(39, 13)


,Postal Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Pub,Neighborhood,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",Fast Food Restaurant,Sandwich Place,Burrito Place,Fish & Chips Shop,Steakhouse,Pub,Sushi Restaurant,Intersection,Food & Drink Shop,Italian Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
5,M4P,Central Toronto,Davisville North,Sandwich Place,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Gym,Park,Donut Shop,Eastern European Restaurant,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Miscellaneous Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Pet Store,Park
7,M4S,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Gym,Italian Restaurant,Coffee Shop,Seafood Restaurant,Diner
8,M4T,Central Toronto,"Moore Park, Summerhill East",Playground,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Sushi Restaurant,Supermarket,Vietnamese Restaurant,Light Rail Station,Sports Bar,Liquor Store,Restaurant,American Restaurant


<h3>Cluster the Areas</h3>

Run k-means to cluster the Toronto areas into 5 clusters.

In [122]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Postal Code", "Borough", "Neighborhoods"],axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([3, 3, 3, 3, 0, 3, 3, 3, 1, 3], dtype=int32)

In [123]:
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("Postal Code"), on="Postal Code")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Pub,Bakery,Theater,Restaurant,Breakfast Spot,Café,Hotel,Chocolate Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Sushi Restaurant,Bank,Beer Bar,Sandwich Place,Restaurant,Burger Joint,Burrito Place,Café,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Café,Restaurant,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Creperie,Lingerie Store,Moroccan Restaurant,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Neighborhood,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio


In [124]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Neighborhood,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Gym,Hotel,Thai Restaurant,Deli / Bodega,Clothing Store,Sushi Restaurant,Bookstore
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,2,Thai Restaurant,Café,Mexican Restaurant,Fast Food Restaurant,Bakery,Flea Market,Speakeasy,Italian Restaurant,Fried Chicken Joint,Arts & Crafts Store
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Pub,Bakery,Theater,Restaurant,Breakfast Spot,Café,Hotel,Chocolate Shop
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Café,Sandwich Place,Coffee Shop,Park,Pub,Middle Eastern Restaurant,BBQ Joint,Donut Shop,History Museum,Burger Joint
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,3,Gift Shop,Bookstore,Restaurant,Bar,Movie Theater,Italian Restaurant,Cuban Restaurant,Dog Run,Dessert Shop,Eastern European Restaurant
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Gym,Italian Restaurant,Coffee Shop,Seafood Restaurant,Diner
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,3,Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore,Restaurant,Bakery,Dessert Shop,Chinese Restaurant,Pub
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,3,Pizza Place,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Food,French Restaurant,Fish & Chips Shop,Dessert Shop
29,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Playground,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


**Finally, let's visualize the resulting clusters**

In [126]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>

**Cluster 1**

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,0,Health Food Store,Pub,Neighborhood,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio


**Cluster 2**

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gym,Hotel,Thai Restaurant,Deli / Bodega,Clothing Store,Sushi Restaurant,Bookstore


**Cluster 3**

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,2,Thai Restaurant,Café,Mexican Restaurant,Fast Food Restaurant,Bakery,Flea Market,Speakeasy,Italian Restaurant,Fried Chicken Joint,Arts & Crafts Store


**Cluster 4**

In [130]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Park,Pub,Bakery,Theater,Restaurant,Breakfast Spot,Café,Hotel,Chocolate Shop
24,Central Toronto,3,Café,Sandwich Place,Coffee Shop,Park,Pub,Middle Eastern Restaurant,BBQ Joint,Donut Shop,History Museum,Burger Joint
25,West Toronto,3,Gift Shop,Bookstore,Restaurant,Bar,Movie Theater,Italian Restaurant,Cuban Restaurant,Dog Run,Dessert Shop,Eastern European Restaurant
26,Central Toronto,3,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Gym,Italian Restaurant,Coffee Shop,Seafood Restaurant,Diner
27,Downtown Toronto,3,Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore,Restaurant,Bakery,Dessert Shop,Chinese Restaurant,Pub
28,West Toronto,3,Pizza Place,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Food,French Restaurant,Fish & Chips Shop,Dessert Shop
29,Central Toronto,3,Playground,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
30,Downtown Toronto,3,Café,Coffee Shop,Vietnamese Restaurant,Bakery,Mexican Restaurant,Bar,Vegetarian / Vegan Restaurant,Gaming Cafe,Dessert Shop,Filipino Restaurant
31,Central Toronto,3,Coffee Shop,Pub,Sushi Restaurant,Supermarket,Vietnamese Restaurant,Light Rail Station,Sports Bar,Liquor Store,Restaurant,American Restaurant
32,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Plane,Coffee Shop,Sculpture Garden


**Cluster 5**

In [131]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Clothing Store,Coffee Shop,Yoga Studio,Miscellaneous Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Pet Store,Park
10,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Brewery,Scenic Lookout,Sporting Goods Shop,Italian Restaurant,Restaurant,Fried Chicken Joint


<h3>Observations</h3>

Most of the neighborhoods fall into Cluster 4 which are mostly business areas with cafe, parks, pub etc. Cluster 3 has Thai restaurants, cafe, etc, Cluster 1 has Health food store, pubs, etc, Cluster 2 has coffee shops, cafe, gyms, etc, and lastly Cluster 5 has clothing stores, coffee shop as top venues.